In [14]:
import torch
import torch.nn as nn
import math

import matplotlib.pyplot as plt

from datasets.openml_datasets import OpenML_Dataset
from datasets.torch_datasets import Torch_Dataset
from utils.weakener import Weakener
from models.general_model import MLP
from utils.losses import PartialLoss,LBLoss,EMLoss,OSLCELoss,OSLBrierLoss,CELoss
from utils.trainig_testing import train_model,evaluate_model,train_and_evaluate

In [7]:
Data = Torch_Dataset('mnist', batch_size=16)
Weak = Weakener(Data.num_classes)
#Weak.generate_M(model_class='pll')

In [8]:
Weak.generate_M('pll',pll_p=0.5)
Weak.M[:5,:],Weak.Z[:5,:]

(array([[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.00217014, 0.00217014],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.00217014, 0.        , 0.00217014],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.00217014, 0.00217014, 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.00195312, 0.00195312, 0.00195312],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.00217014, 0.        , 0.        , 0.00217014]]),
 array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 1]]))

In [9]:
train_X,train_y,test_X,test_y =  Data.get_data()
print(train_X.shape)
Weak.generate_weak(train_y)

torch.Size([60000, 784])


(tensor([717, 934, 166,  ..., 998,  98,  71], dtype=torch.int32),
 tensor([[1., 0., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 1.],
         [0., 0., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 1., 1., 1.]], dtype=torch.float64))

In [10]:
train_y[:5,:],Weak.z[:5],Weak.w[:5,:]

(tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]),
 tensor([717, 934, 166, 457, 584], dtype=torch.int32),
 tensor([[1., 0., 1., 1., 0., 1., 1., 0., 0., 0.],
         [1., 1., 1., 0., 1., 1., 0., 0., 0., 1.],
         [0., 0., 1., 0., 1., 0., 1., 1., 1., 1.],
         [0., 1., 1., 1., 0., 1., 0., 0., 1., 1.],
         [1., 0., 0., 1., 0., 1., 0., 0., 1., 1.]], dtype=torch.float64))

In [15]:
mlp_feature = MLP(Data.num_features, [Data.num_features, Data.num_features, Data.num_features], Data.num_classes, dropout_p = 0.0, bn = False, seed = 1,
                  layer_init = lambda x: nn.init.kaiming_uniform_(x, a=math.sqrt(5)))

In [18]:
class mlp_feature(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(mlp_feature, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out = x.view(x.size(0), -1)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        feature = self.relu3(out)
        out = self.fc4(out)
        return feature, out

In [44]:
mlp_feat = mlp_feature(Data.num_features, Data.num_features, Data.num_classes)


In [46]:
mlp_feat.state_dict()

OrderedDict([('fc1.weight',
              tensor([[ 0.0160,  0.0357,  0.0259,  ...,  0.0346,  0.0098, -0.0007],
                      [-0.0038,  0.0186, -0.0336,  ..., -0.0220, -0.0235, -0.0181],
                      [ 0.0314,  0.0101, -0.0083,  ...,  0.0088,  0.0324,  0.0314],
                      ...,
                      [-0.0005, -0.0201, -0.0312,  ...,  0.0286,  0.0098, -0.0282],
                      [-0.0321, -0.0128, -0.0284,  ..., -0.0145,  0.0013, -0.0034],
                      [ 0.0264, -0.0247,  0.0258,  ...,  0.0247,  0.0001,  0.0357]])),
             ('fc1.bias',
              tensor([-2.5243e-02,  3.1946e-02, -2.4145e-02, -1.5259e-02,  1.0686e-02,
                      -7.4537e-03, -3.3765e-02, -3.4047e-02,  3.6584e-03, -1.4164e-02,
                      -9.2037e-03,  9.8986e-03, -1.8367e-02,  1.4549e-02, -2.0212e-02,
                       1.5605e-02, -2.8576e-02,  1.5315e-02, -2.2751e-02,  1.5439e-02,
                      -1.0001e-02,  3.1898e-02,  3.3839e-02, -1.

In [42]:
state_dict.keys()

odict_keys(['fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias', 'fc4.weight', 'fc4.bias'])

In [25]:
state_dict = torch.load('results/mnist_random')

# Extract the weights for each layer from the state dict
weights = {}
for key, value in state_dict.items():
    if 'weight' in key:
        weights[key] = value

# Print the extracted weights
print(weights)

{'fc1.weight': tensor([[ 0.0172, -0.0161, -0.0075,  ...,  0.0059, -0.0047,  0.0017],
        [-0.0267, -0.0219, -0.0302,  ..., -0.0029,  0.0012, -0.0163],
        [ 0.0275, -0.0093, -0.0333,  ..., -0.0138,  0.0055, -0.0328],
        ...,
        [-0.0015, -0.0159, -0.0231,  ..., -0.0109,  0.0195, -0.0093],
        [ 0.0065,  0.0274, -0.0327,  ...,  0.0053,  0.0243,  0.0175],
        [-0.0229, -0.0293,  0.0004,  ..., -0.0110,  0.0073, -0.0074]]), 'fc2.weight': tensor([[ 9.1917e-03,  1.0989e-02,  2.6633e-02,  ..., -9.0834e-03,
          1.9386e-05, -3.3072e-02],
        [-9.9623e-03,  3.4387e-03,  3.2542e-02,  ..., -6.5294e-03,
         -1.2326e-02,  1.0187e-02],
        [ 3.4065e-02,  1.8922e-02,  5.3480e-03,  ..., -4.2586e-03,
         -7.5498e-03,  2.1564e-02],
        ...,
        [-1.3778e-02, -3.2532e-02,  3.2409e-02,  ...,  7.7200e-03,
          3.2663e-02, -2.3619e-02],
        [-1.5580e-02,  1.9131e-03,  4.5237e-03,  ...,  2.5431e-02,
         -3.0175e-03, -3.1466e-02],
        

In [36]:
weights.keys()

dict_keys(['fc1.weight', 'fc2.weight', 'fc3.weight', 'fc4.weight'])

In [32]:
weights['fc2.weight'].shape

torch.Size([784, 784])

In [33]:
list(mlp_feat.parameters())[1].shape

torch.Size([784])

In [35]:
len(list(mlp_feat.parameters()))

8

In [ ]:
#list(mlp_feat.parameters())[0] = weights['fc1.weight']

In [2]:
Data = Torch_Dataset('mnist')
train_x, train_y, test_x, test_y = Data.get_data()
# WL stores processes relative to the Weakening process
Weak = Weakener(Data.num_classes)
Weak.generate_M(model_class='pll')

In [3]:
z, w = Weak.generate_weak(train_y)

In [4]:
Data.include_weak(Weak.w)

In [5]:
trainloader, testloader = Data.get_dataloader()

In [6]:
mlp = MLP(Data.num_features,[Data.num_features,Data.num_features,Data.num_features],Data.num_classes,dropout_p=0.3)
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.001)
loss_fn = OSLCELoss()

#mlp,results = train_and_evaluate(mlp,trainloader,testloader,optimizer,loss_fn,num_epochs=10)

In [11]:
import pickle
with open('results/mnist_random.pkl', 'rb') as file:
    models = pickle.load(file)


#mlp = model_data['model']
#results = model_data['results']

ModuleNotFoundError: No module named 'models.mlp'

In [3]:
model_data

NameError: name 'model_data' is not defined

In [7]:
MLP()

TypeError: __init__() missing 3 required positional arguments: 'input_size', 'hidden_sizes', and 'output_size'